In [1]:
import pandas as pd, numpy as np, re, ast, json, os
from tqdm.auto import tqdm
from gensim import downloader as api
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from pathlib import Path

res_df = pd.read_csv("./gold_samples/resumes_samples.csv")   
jd_df  = pd.read_csv("./gold_samples/job_desc_sampled.csv")  

assert len(res_df)==50 and len(jd_df)==50, "Expecting 50 resumes & 50 JDs"

gold_file = Path("gold_res.txt")
row_pat   = re.compile(r"R(\d+)\s*:\s*\[(.*)\]")
jd_pat    = re.compile(r"JD\d+")

gold_dict = {}
with gold_file.open() as fh:
    for raw in fh:
        m = row_pat.match(raw.strip())
        if not m:
            continue
        rid      = f"R{m.group(1)}"
        jd_list  = jd_pat.findall(m.group(2))
        gold_dict[rid] = jd_list

assert len(gold_dict) == 50, "Gold file should contain 50 rows (R1…R50)"

/Users/chiragjain/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tok_re = re.compile(r"[A-Za-z\d\-\+#]+")

def preprocess(text:str) -> str:
    return " ".join(tok_re.findall(text.lower()))

res_texts = res_df["resume_text"].apply(preprocess).tolist()
jd_texts  = jd_df["job_description"].apply(preprocess).tolist()

vectorizer = TfidfVectorizer(token_pattern=r"[A-Za-z\d\-\+#]+",
                             lowercase=False,      
                             min_df=1, max_df=0.9) 

corpus = res_texts + jd_texts
vectorizer.fit(corpus)
idf     = vectorizer.idf_         
vocab   = vectorizer.vocabulary_  

# map idf to token
idf_dict = {tok: idf[idx] for tok, idx in vocab.items()}

In [3]:
print("Downloading GloVe 300‑d word vectors … (one‑time ~376 MB)")
wv = api.load("word2vec-google-news-300")  
dim = wv.vector_size

[--------------------------------------------------] 1.4% 23.4/1662.8MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[==------------------------------------------------] 5.7% 95.1/1662.8MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[====----------------------------------------------] 8.7% 144.0/1662.8MB downloaded

...

[================================================--] 96.5% 1604.2/1662.8MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[================================================--] 97.8% 1626.4/1662.8MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [4]:
def doc_vector(text:str) -> np.ndarray:
    tokens = text.split()
    if not tokens:
        return np.zeros(dim)
    vecs, weights = [], []
    for tok in tokens:
        if tok in wv:
            vecs.append(wv[tok])
            weights.append(idf_dict.get(tok, np.median(idf)))
    if not vecs:
        return np.zeros(dim)
    vecs = np.vstack(vecs)
    weights = np.array(weights)[:, None]       # shape (n,1)
    weighted = (vecs * weights).sum(axis=0) / weights.sum()
    return weighted

print("Building document vectors …")
res_emb = np.vstack([doc_vector(txt) for txt in tqdm(res_texts, desc="Resumes ")])
jd_emb  = np.vstack([doc_vector(txt) for txt in tqdm(jd_texts,  desc="Job Descs")])

# L2‑normalise for cosine sim
res_emb = res_emb / np.linalg.norm(res_emb, axis=1, keepdims=True)
jd_emb  = jd_emb  / np.linalg.norm(jd_emb,  axis=1, keepdims=True)

Building document vectors …


Job Descs: 100%|███████████████████████████████| 50/50 [00:00<00:00, 158.08it/s]


In [5]:
sim_matrix = cosine_similarity(res_emb, jd_emb)

topk = 10
retrieved = {}         
for i, sim_row in enumerate(sim_matrix):
    top_idx = np.argsort(sim_row)[-topk:][::-1]            # best→worst
    top_jds = jd_df.iloc[top_idx]["jd_id"].tolist()
    retrieved[f"R{i+1}"] = top_jds

In [6]:
def precision_at_k(pred, gold, k=10):
    return len(set(pred[:k]) & set(gold)) / k

def recall_at_k(pred, gold, k=10):
    return len(set(pred[:k]) & set(gold)) / len(gold)

def mrr_at_k(pred, gold, k=10):
    for rank, jd in enumerate(pred[:k], 1):
        if jd in gold:
            return 1.0 / rank
    return 0.0

def ndcg_at_k(pred, gold, k=10):
    dcg = 0.0
    for rank, jd in enumerate(pred[:k], 1):
        if jd in gold:
            dcg += 1.0 / np.log2(rank + 1)
    idcg = sum(1.0 / np.log2(r + 1) for r in range(1, min(len(gold), k) + 1))
    return dcg / idcg if idcg else 0.0

def topk_accuracy(pred, gold, k=10):
    return int(bool(set(pred[:k]) & set(gold)))

In [7]:
P,R,ACC,MRR,NDCG = [],[],[],[],[]
for rid,preds in retrieved.items():
    gold = gold_dict[rid]
    P.append( precision_at_k(preds,gold) )
    R.append( recall_at_k(preds,gold) )
    ACC.append( topk_accuracy(preds,gold) )
    MRR.append( mrr_at_k(preds,gold) )
    NDCG.append( ndcg_at_k(preds,gold) )

metrics = pd.DataFrame({
    "Metric": ["Precision@10","Recall@10","Top‑10 accuracy","MRR@10","NDCG@10"],
    "Value" : [np.mean(P), np.mean(R), np.mean(ACC), np.mean(MRR), np.mean(NDCG)]
})

print("\n===  Retrieval quality (average over 50 resumes)  ===")
print(metrics.to_string(index=False, float_format=lambda x:f"{x:0.4f}"))


===  Retrieval quality (average over 50 resumes)  ===
         Metric  Value
   Precision@10 0.3200
      Recall@10 0.3200
Top‑10 accuracy 1.0000
         MRR@10 0.7657
        NDCG@10 0.3709


In [ ]:
# ===  Retrieval quality (average over 50 resumes)  ===
#          Metric  Value
#    Precision@10 0.2840
#       Recall@10 0.2840
# Top‑10 accuracy 0.9400
#          MRR@10 0.5642
#         NDCG@10 0.3018

In [10]:
for rid in sorted(retrieved, key=lambda s:int(s[1:])):
    print(f"\n{rid} → {retrieved[rid]}")


R1 → ['JD38', 'JD4', 'JD7', 'JD18', 'JD6', 'JD31', 'JD8', 'JD29', 'JD36', 'JD5']

R2 → ['JD1', 'JD5', 'JD48', 'JD49', 'JD46', 'JD10', 'JD50', 'JD4', 'JD7', 'JD38']

R3 → ['JD1', 'JD5', 'JD49', 'JD48', 'JD46', 'JD50', 'JD4', 'JD10', 'JD7', 'JD38']

R4 → ['JD1', 'JD5', 'JD48', 'JD49', 'JD46', 'JD50', 'JD10', 'JD4', 'JD7', 'JD38']

R5 → ['JD5', 'JD1', 'JD4', 'JD49', 'JD7', 'JD46', 'JD48', 'JD50', 'JD38', 'JD10']

R6 → ['JD7', 'JD38', 'JD4', 'JD5', 'JD31', 'JD50', 'JD8', 'JD36', 'JD18', 'JD49']

R7 → ['JD7', 'JD38', 'JD24', 'JD4', 'JD31', 'JD49', 'JD8', 'JD11', 'JD6', 'JD46']

R8 → ['JD7', 'JD38', 'JD4', 'JD11', 'JD31', 'JD20', 'JD49', 'JD13', 'JD24', 'JD14']

R9 → ['JD6', 'JD38', 'JD7', 'JD31', 'JD8', 'JD18', 'JD34', 'JD17', 'JD27', 'JD4']

R10 → ['JD48', 'JD5', 'JD49', 'JD1', 'JD46', 'JD50', 'JD4', 'JD7', 'JD38', 'JD8']

R11 → ['JD20', 'JD7', 'JD38', 'JD13', 'JD6', 'JD49', 'JD31', 'JD8', 'JD29', 'JD4']

R12 → ['JD7', 'JD47', 'JD17', 'JD31', 'JD38', 'JD9', 'JD36', 'JD40', 'JD32', 'JD27']